In [1]:
import sys 
sys.path.append('..')
# sys.path.insert(0,'/home/ildefog/pydyno')
import pydyno
print(pydyno.__file__)
from pydyno.discretization.pysb_discretize import PysbDomPath
from pydyno.seqanalysis import SeqAnalysis
from pydyno.visualize_simulations import VisualizeSimulations
from pydyno.visualize_discretization import visualization_path
%matplotlib inline
import random
import h5py
from pydyno.dominant_path_frequency_analysis import FreqAnalysis
import sys 
# sys.path.append('/Users/geenaildefonso/Projects/NERM')
sys.path.append('../')
from model_files.nerm import model as model
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
# from necro_pydream_model_file_8920 import model
from pysb.simulator import ScipyOdeSimulator
import numpy as np
from scipy.stats import norm,uniform
import seaborn as sns
from matplotlib import pyplot as plt
import random
import os
from pylab import *
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
from pysb.util import alias_model_components
from scipy import stats
from pysb.simulator import SimulationResult
import pandas as pd
import seaborn as sns

/home/ildefog/.local/lib/python3.7/site-packages/pydyno-0.1.1+94.g9b93be5-py3.7-linux-x86_64.egg/pydyno/__init__.py


In [2]:
pars = np.load('../necro_pars10k_allclusmod.npy')

In [ ]:
for i,sp in enumerate(model.species):
    print(i,":",sp)

In [3]:
m = model.components
tspan = np.linspace(0, 1800, 500)

In [4]:
solverwt0 = ScipyOdeSimulator(model, tspan=tspan, verbose = True)
result100 = solverwt0.run(param_values=pars, num_processors = 20)
# result100.save('sim_nerm_allpars_tnf100.h5')

2021-11-20 18:21:27.898 - pysb.simulator.scipyode - DEBUG - [model_files.nerm] Simulator created
2021-11-20 18:21:28.671 - pysb.simulator.scipyode - DEBUG - [model_files.nerm] Equation mode set to "cython"
2021-11-20 18:21:28.781 - pysb.simulator.scipyode - INFO - [model_files.nerm] Simulation(s) started
2021-11-20 18:21:29.323 - pysb.simulator.scipyode - DEBUG - [model_files.nerm] Multi-processor (parallel) mode using 20 processes
2021-11-20 18:50:15.168 - pysb.simulator.scipyode - INFO - [model_files.nerm] All simulation(s) complete
2021-11-20 18:50:15.169 - pysb.simulator.scipyode - DEBUG - [model_files.nerm] SimulationResult constructor started
2021-11-20 18:50:20.608 - pysb.simulator.scipyode - DEBUG - [model_files.nerm] SimulationResult constructor finished


In [ ]:
for i,sp in enumerate(model.species):
    print(i,":",sp)

In [5]:
dp_nerm_100mlkl = PysbDomPath(model, result100)

In [ ]:
signs_nerm_100mlkl, paths_nerm_100mlkl = dp_nerm_100mlkl.get_path_signatures('s36', 'production', 
                                                                 depth=30, dom_om=0.5, num_processors = 20)

In [ ]:
signs_nerm_100mlkl.dissimilarity_matrix()

In [ ]:
signs_nerm_100mlkl.silhouette_score_spectral_range(range(2,10),num_processors = 20)

In [ ]:
signs_nerm_100mlkl.silhouette_score_agglomerative_range(10 ,num_processors=20) # n_clusters best silhouette score: 3

In [ ]:
signs_nerm_100mlkl.agglomerative_clustering(2)

In [ ]:
signs_nerm_100mlkl.spectral_clustering(8)

In [ ]:
sorted_label_idxs = np.argsort(signs_nerm_100.labels)
sorted_diss = signs_nerm_100.diss[sorted_label_idxs, :]
sorted_diss = sorted_diss[:, sorted_label_idxs]
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    if not ax:
        ax = plt.gca()
    # Plot the heatmap
    im = ax.imshow(data, **kwargs)
    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
#     ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)
    return im, cbar
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
im, cbar = heatmap(sorted_diss, range(10000), range(10000), ax=ax,
                   cmap="YlGn", cbarlabel="distance")
ax.figure.axes[-1].yaxis.label.set_size(14)
plt.xlabel('signatures', fontsize = 14)
plt.ylabel('signatures', fontsize = 14)
fig.tight_layout()
# plt.savefig('heatmap_clusters_necro_pydream_5chns_10k_5720_10tnf_aug.pdf')
plt.show()

In [ ]:
plt.figure(figsize = (20,10))
signs_nerm_100.plot_sequences(type_fig='modal')
plt.show()

In [ ]:
    def plot_sequences(self, type_fig='modal', plot_all=False, title='', dir_path='', sort_seq=None):
    if plot_all:
            cluster_labels = np.zeros(len(self.seq_analysis.sequences), dtype=np.int)
        else:
            # Check that the sequences has been clustered
            if self.seq_analysis.labels is None:
                raise Exception('Cluster the sequences first')
            cluster_labels = self.seq_analysis.labels

        clusters = set(cluster_labels)
        if -1 in clusters:
            clusters = list(clusters)[:-1]  # this is to not plot the signatures that can't be clustered :( from hdbscan
        else:
            clusters = list(clusters)
        n_rows = int(math.ceil(len(clusters) / 3.))
        if len(clusters) == 1:
            f, axs = plt.subplots(n_rows, 1, sharex=True, figsize=(8, 6))
            axs = [axs]
        elif len(clusters) == 2:
            f, axs = plt.subplots(n_rows, 2, sharex=True, figsize=(8, 6))
        else:
            f, axs = plt.subplots(n_rows, 3, sharex=True, figsize=(8, 6))
            f.subplots_adjust(hspace=.6, wspace=.4)
            axs = axs.reshape(n_rows * 3)

            plots_off = (n_rows * 3) - len(clusters)
            for i in range(1, plots_off + 1):
                axs[-i].axis('off')

        if type_fig == 'modal':
            self.__modal(cluster_labels, clusters, axs, n_rows)

        elif type_fig == 'trajectories':
            self.__trajectories(cluster_labels, clusters, axs, n_rows, sort_seq)

        elif type_fig == 'entropy':
            self.__entropy(cluster_labels, clusters, f, axs, n_rows)

        else:
            raise NotImplementedError('Type of visualization not implemented')

        plt.setp([a.get_xticklabels() for a in f.axes[:-3]], visible=False)
        plt.suptitle(title)
        f.text(0.5, 0.04, 'Time', ha='center')
        final_path = os.path.join(dir_path, type_fig + '.png')
        plt.savefig(final_path, bbox_inches='tight', format='png')
        # plt.close('all')

        return

In [ ]:
signs_nerm_100.plot_sequences(type_fig='trajectories')

In [ ]:
vt = VisualizeSimulations(model, sim_results=result100, 
                          clusters=signs_nerm_100.labels)

In [ ]:
vt.plot_cluster_dynamics(components=['MLKLa_obs'])

In [ ]:
cluster_label_idxs = VisualizeSimulations.check_clusters_arg(signs_nerm_100.labels, nsims=10000)

In [ ]:
pars_cluster_0 = result100.param_values[cluster_label_idxs[0]]
pars_cluster_1 = result100.param_values[cluster_label_idxs[1]]
pars_cluster_2 = result100.param_values[cluster_label_idxs[2]]
pars_cluster_3 = result100.param_values[cluster_label_idxs[3]]
pars_cluster_4 = result100.param_values[cluster_label_idxs[4]]
pars_cluster_5 = result100.param_values[cluster_label_idxs[5]]
pars_cluster_6 = result100.param_values[cluster_label_idxs[6]]
pars_cluster_7 = result100.param_values[cluster_label_idxs[7]]
print(len(pars_cluster_0))
print(len(pars_cluster_1))
print(len(pars_cluster_2))
print(len(pars_cluster_3))
print(len(pars_cluster_4))
print(len(pars_cluster_5))
print(len(pars_cluster_6))
print(len(pars_cluster_7))

In [ ]:
rep_signatures=signs_nerm_100.seq_representativeness(method='centrality', clus_labels=signs_nerm_100.labels)

In [ ]:
print(rep_signatures[0])

In [ ]:
from pydyno.visualize_discretization import visualization_seq_paths
visualization_seq_paths(result100, 3528, signs_nerm_100)
# plt.savefig('visualization_seq_paths_clus0_sigs100n_3528.pdf')

In [ ]:
from IPython.display import Image
path_to_show = '57'
visualization_path(model, paths_prod[path_to_show], 's24', 'production', f'path_{path_to_show}.png')
Image(filename=f'path_{path_to_show}.pdf') 

In [ ]:
visualization_path(model, paths_nerm_100[57], type_analysis='production', 
                   filename='cluster0100tnf_CII_s24_prod_1om_depth15_path57_10k_5720_112021.pdf')

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

X_embedded = TSNE(n_components=3, metric='precomputed', perplexity=50,
                  early_exaggeration=30).fit_transform(signs_nerm_100.diss)

In [ ]:
colors = ['red', 'blue', 'black', 'green', 'cyan', 'yellow', 'orange', 'purple']

In [ ]:
cluster0 = signs_nerm_100.labels == 0
cluster1 = signs_nerm_100.labels == 1
cluster2 = signs_nerm_100.labels == 2
cluster3 = signs_nerm_100.labels == 3
cluster4 = signs_nerm_100.labels == 4
cluster5 = signs_nerm_100.labels == 5
cluster6 = signs_nerm_100.labels == 6
cluster7 = signs_nerm_100.labels == 7

plt.scatter(X_embedded[cluster0, 0], X_embedded[cluster0, 1], c=colors[0], label='clus_0')
plt.scatter(X_embedded[cluster1, 0], X_embedded[cluster1, 1], c=colors[1], label='clus_1')
plt.scatter(X_embedded[cluster2, 0], X_embedded[cluster2, 1], c=colors[2], label='clus_2')
plt.scatter(X_embedded[cluster3, 0], X_embedded[cluster3, 1], c=colors[3], label='clus_3')
plt.scatter(X_embedded[cluster4, 0], X_embedded[cluster4, 1], c=colors[4], label='clus_4')
plt.scatter(X_embedded[cluster5, 0], X_embedded[cluster5, 1], c=colors[5], label='clus_5')
plt.scatter(X_embedded[cluster6, 0], X_embedded[cluster6, 1], c=colors[6], label='clus_6')
plt.scatter(X_embedded[cluster7, 0], X_embedded[cluster7, 1], c=colors[7], label='clus_7')

plt.legend(bbox_to_anchor=(1.25,1), borderaxespad=0)
plt.xlabel('tSNE_1')
plt.ylabel('tSNE_2')
plt.tight_layout()
plt.title('TSNE plot for Cluster Dynamics')
# plt.savefig('necro_5720_10k_100tnf_sequences_cluster.pdf')
plt.show()